### Item 13: Take Advantage of Each Block in try/except/else/finally

* There are four distinct times that you may want to take action during exception handling in Python.
* These are captured in the functionality of `try`, `except`, `else`, and `finally` blocks.


* Each block serves a unique purpose in the compound statement, and their various combinations are useful.
    * See `Item 51`: Define a Root Exception to Insulate Callers from APIs

#### Finally Blocks

* Use try/finally when you want exceptions to propagate up, or want to run cleanup code even when exceptions occur.


* One common usage of try/finally is for reliably closing file handles.
    * See `Item 43`: "Consider contextlib and with Statemets for Reusable try/finally Behavior"

In [ ]:
# random JSON data
input_txt = "../data/random_data.txt"

handle = open(input_txt)  # May raise IOError
try:
    data = handle.read()  # May raise UnicodeDecodeError
finally:
    handle.close()  # Always runs after try:

* Any `exception` raised by the `read` method will always propagate up to the calling code.
* Yet the `close` method of handle is also guaranteed to run in the `finally` block.
* You must call `open` before the `try` block because `exception` that occur when opening the file (like `IOError` if the file does not exist) should skip the `finally` block.

#### Else block

* Use `try/except/else` to make it clear which exceptions will be handled by your code and which exceptions will propagate up.
* When the `try` block doesn't raise an exception, the `else` block willl run.
* the `else` block helps you minimize the amount of code in the `try` block and improves readability.

In [ ]:
import json

def load_json_key(data, key):
    try:
        result_dict = json.loads(data)  # May raise ValueError
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key]  # May raise KeyError

In [ ]:
data

In [ ]:
key = 'popup'
load_json_key(data, key)

* If the data isn't JSON, then decoding with json.loads will raise a ValueError.
* The exception is caught by the except block and hendled.
* If decoding is successful, then the key lookup will occur i the else block.
* if the key lookup rasises any exceptions, they will propagate up to the caller because they are outside the try block.
* The else clause ensures that what follows the try/except is visually distinguished from the except block.
* This makes the exception propagation behavior clear.

#### Everything together

Use try/except/else/finally when you want to do it all in one compound statement.

In [ ]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')  # May raise IOError
    try:
        data = handle.read()  # May raise UnicodeDecodeError
        op = json.loads(data)  # May raise ValueError
        value = (
            op['numerator'] /
            op['denominator'])  # May raise ZeroDivisionError
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)  # May raise IOError
        return value
    finally:
        handle.close()  # Always runs      

* This layout is especially useful because all of the blocks work together in intuitive ways.

### Things to Remember

* The `try/finally` compound statement lets you run cleanup code regardless of whether exceptions were raised in the try block.
* The `else` block helps you minimize the amount of code in try blocks and visually distinguish the success case from the `try/except` blocks.
* An `else` block can be used to perform additional actions after a successful try block but before common cleanup in a `finally` block.